In [21]:

#Input data we are making a electricity system with 3 generators and 3 demands and 8 buses in total
#they should not be slack buses
buses = {
    'Bus1': {'type': 'generator'},
    'Bus2': {'type': 'generator'},
    'Bus3': {'type': 'generator'},
    'Bus4': {'type': 'generator'},
    'Bus5': {'type': 'load'},
    'Bus6': {'type': 'generator'},
    'Bus7': {'type': 'load'},
    'Bus8': {'type': 'load'},
}
generators = {
    'Gen1': {'bus': 'Bus1', 'capacity': 300, 'cost': 20},
    'Gen2': {'bus': 'Bus2', 'capacity': 150, 'cost': 10},
    'Gen3': {'bus': 'Bus3', 'capacity': 600, 'cost': 30},
    'Gen4': {'bus': 'Bus4', 'capacity': 0, 'cost': 0},
    'Gen5': {'bus': 'Bus6', 'capacity': 0, 'cost': 0},
}
loads = {
    'Load5': {'bus': 'Bus5', 'demand': 80},
    'Load7': {'bus': 'Bus7', 'demand': 120},
    'Load8': {'bus': 'Bus8', 'demand': 150},
}
lines = {
    ('Bus1', 'Bus4'): {'capacity': 50},
    ('Bus1', 'Bus2'): {'capacity': 50},
    ('Bus2', 'Bus3'): {'capacity': 70},
    ('Bus2', 'Bus5'): {'capacity': 50},
    ('Bus3', 'Bus6'): {'capacity': 90},
    ('Bus4', 'Bus7'): {'capacity': 80},
    ('Bus5', 'Bus7'): {'capacity': 70},    
    ('Bus5', 'Bus6'): {'capacity': 65},
    ('Bus6', 'Bus8'): {'capacity': 50},  
    ('Bus7', 'Bus8'): {'capacity': 30},
}  
#Cost for increasing line capacities
line_upgrade_cost = 5  # Cost per unit increase in line capacity 

#write this model 




In [15]:
import gurobipy as gp
from gurobipy import GRB
# Create a new model
model = gp.Model("electricity_system")

# Create variables for generator outputs
gen_vars = {}
for gen, data in generators.items():
    gen_vars[gen] = model.addVar(lb=0, ub=data['capacity'], name=f"GenOutput_{gen}")

# Create variables for line flows and capacity upgrades (one per undirected line)
line_vars = {}
line_upgrade_vars = {}
for (bus_from, bus_to), data in lines.items():
    cap = data['capacity']
    # single flow var for the undirected line: allow negative values for reverse flow
    line_vars[(bus_from, bus_to)] = model.addVar(name=f"LineFlow_{bus_from}_{bus_to}")
    # single upgrade var (non-negative) for that same undirected line
    line_upgrade_vars[(bus_from, bus_to)] = model.addVar(name=f"LineUpgrade_{bus_from}_{bus_to}")

# Add capacity constraints: |flow| <= capacity + upgrade
for (bus_from, bus_to), data in lines.items():
    cap = data['capacity']
    f = line_vars[(bus_from, bus_to)]
    u = line_upgrade_vars[(bus_from, bus_to)]
    model.addConstr(f <= cap + u, name=f"CapPos_{bus_from}_{bus_to}")
    model.addConstr(f >= - (cap + u), name=f"CapNeg_{bus_from}_{bus_to}")

# Create constraints for power balance at each bus
for bus, data in buses.items():
    inflow = gp.LinExpr()
    outflow = gp.LinExpr()
    generation = gp.LinExpr()
    demand = gp.LinExpr()
    
    # Sum inflows and outflows from lines
    for (bus_from, bus_to), var in line_vars.items():
        if bus_to == bus:
            inflow += var
        if bus_from == bus:
            outflow += var
    
    # Sum generation at this bus
    for gen, gen_data in generators.items():
        if gen_data['bus'] == bus:
            generation += gen_vars[gen]
    
    # Sum demand at this bus
    for load, load_data in loads.items():
        if load_data['bus'] == bus:
            demand += load_data['demand']
    
    # Power balance constraint
    model.addConstr(generation + inflow == demand + outflow, name=f"PowerBalance_{bus}")
# Set objective: minimize generation cost and line upgrade cost
gen_cost = gp.quicksum(gen_vars[gen] * data['cost'] for gen, data in generators.items())
upgrade_cost = gp.quicksum(line_upgrade_vars[(bus_from, bus_to)] * line_upgrade_cost for (bus_from, bus_to) in lines.keys())
model.setObjective(gen_cost + upgrade_cost, GRB.MINIMIZE)
# Optimize the model
model.optimize()
# Print the results
if model.status == GRB.OPTIMAL: 
    #Print the cost of line capacity upgrades
    print("Line capacity upgrade cost results:")
    for (bus_from, bus_to), var in line_upgrade_vars.items():
        if var.x > 0:
            print(f"Upgrade on line {bus_from} to {bus_to}: {var.x} MW")
            

    print("\nOptimal Generation Outputs:")
    for gen, var in gen_vars.items():
        print(f"{gen}: {var.x} MW")
    
    print("\nOptimal Line Flows:")
    for (bus_from, bus_to), var in line_vars.items():
        print(f"Flow from {bus_from} to {bus_to}: {var.x} MW")
    
    print("\nLine Capacity Upgrades:")
    for (bus_from, bus_to), var in line_upgrade_vars.items():
        if var.x > 0:
            print(f"Upgrade on line {bus_from} to {bus_to}: {var.x} MW")
    
    print(f"\nTotal Cost: {model.objVal}")
    

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 24.6.0 24G325)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 25 columns and 65 nonzeros
Model fingerprint: 0xfbf1337c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+02, 2e+02]
  RHS range        [3e+01, 2e+02]
Presolve removed 15 rows and 7 columns
Presolve time: 0.02s
Presolved: 13 rows, 18 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.875000e+01   0.000000e+00      0s
       8    6.0500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.03 seconds (0.00 work units)
Optimal objective  6.050000000e+03
Line capacity upgrade cost results:
Upgrade on line Bus2 to Bus5: 100.0 MW
Upgrade on line Bus3 to Bus6: 60.0 MW
Upgrade on line Bus6 to Bus8: 100

In [46]:
import gurobipy as gp
from gurobipy import GRB
import math
import random
# Cost for increasing line capacities
# -----------------------
# 20-year settings
# ------------------------
years = range(1, 21)  # Years 1..20

# 20-year load growth factors (2% per year)
load_growth_factor = {}
load_growth_factor = {y: random.uniform(0.5, 1.5) for y in years}


In [ ]:

line_upgrade_cost = 5  # Cost per unit increase in line capacity

yearly_upgrade_budget = 250  # total money per year for all line upgrades

# Value of Lost Load (penalty for shedding)
VOLL = 1000  # €/MW or similar

# ------------------------
# Model
# ------------------------
model = gp.Model("electricity_system_20_years_with_budget")

# Generator outputs per year
gen_vars = {}
for gen, data in generators.items():
    for y in years:
        gen_vars[(gen, y)] = model.addVar(
            lb=0,
            ub=data['capacity'],
            name=f"GenOutput_{gen}_Y{y}"
        )

# Line flows per year (signed: + from bus_from to bus_to)
line_vars = {}
for (bus_from, bus_to), data in lines.items():
    for y in years:
        line_vars[(bus_from, bus_to, y)] = model.addVar(
            lb=-GRB.INFINITY,
            name=f"LineFlow_{bus_from}_{bus_to}_Y{y}"
        )

# Line capacity upgrades per line and year (incremental, cumulative over time)
line_upgrade_vars = {}
for (bus_from, bus_to), data in lines.items():
    for y in years:
        line_upgrade_vars[(bus_from, bus_to, y)] = model.addVar(
            lb=0,
            name=f"LineUpgrade_{bus_from}_{bus_to}_Y{y}"
        )

# Load shedding per load and year (unserved demand)
shed_vars = {}
for load, load_data in loads.items():
    for y in years:
        shed_vars[(load, y)] = model.addVar(
            lb=0,
            name=f"Shed_{load}_Y{y}"
        )

model.update()

# ------------------------
# Line capacity constraints
# For each year: |flow_y| <= base_capacity + sum(upgrades up to year y)
# ------------------------
for (bus_from, bus_to), data in lines.items():
    base_cap = data['capacity']
    for y in years:
        f = line_vars[(bus_from, bus_to, y)]
        # cumulative upgrade up to and including year y
        upgraded_cap = base_cap + gp.quicksum(
            line_upgrade_vars[(bus_from, bus_to, yy)]
            for yy in years if yy <= y
        )
        model.addConstr(
            f <= upgraded_cap,
            name=f"CapPos_{bus_from}_{bus_to}_Y{y}"
        )
        model.addConstr(
            f >= -upgraded_cap,
            name=f"CapNeg_{bus_from}_{bus_to}_Y{y}"
        )

# ------------------------
# Yearly budget constraints for upgrades
# sum_lines (upgrade_cost * upgrade_var[line,y]) <= yearly_upgrade_budget
# ------------------------
for y in years:
    model.addConstr(
        gp.quicksum(
            line_upgrade_cost * line_upgrade_vars[(bus_from, bus_to, y)]
            for (bus_from, bus_to) in lines.keys()
        ) <= yearly_upgrade_budget,
        name=f"UpgradeBudget_Y{y}"
    )

# ------------------------
# Power balance at each bus and year
# generation + net_import + shed = demand
# ------------------------
for y in years:
    for bus, bus_data in buses.items():
        generation = gp.LinExpr()
        net_import = gp.LinExpr()
        demand = gp.LinExpr()
        shed = gp.LinExpr()

        # Generation at this bus in year y
        for gen, gen_data in generators.items():
            if gen_data['bus'] == bus:
                generation += gen_vars[(gen, y)]

        # Net import from incident lines
        for (bus_from, bus_to), _ in lines.items():
            f = line_vars[(bus_from, bus_to, y)]
            if bus == bus_from:
                net_import += -f   # export if f > 0
            elif bus == bus_to:
                net_import += f    # import if f > 0

        # Demand and shedding at this bus in year y
        for load, load_data in loads.items():
            if load_data['bus'] == bus:
                base_demand = load_data['demand']
                demand += base_demand * load_growth_factor[y]
                shed += shed_vars[(load, y)]

        # Power balance
        model.addConstr(
            generation + net_import + shed == demand,
            name=f"PowerBalance_{bus}_Y{y}"
        )

# ------------------------
# Objective: generation cost + upgrade cost + shedding penalty
# ------------------------
gen_cost = gp.quicksum(
    gen_vars[(gen, y)] * generators[gen]['cost']
    for gen in generators
    for y in years
)

upgrade_cost_total = gp.quicksum(
    line_upgrade_cost * line_upgrade_vars[(bus_from, bus_to, y)]
    for (bus_from, bus_to) in lines.keys()
    for y in years
)

shed_cost = gp.quicksum(
    shed_vars[(load, y)] * VOLL
    for load in loads
    for y in years
)

model.setObjective(gen_cost + upgrade_cost_total + shed_cost, GRB.MINIMIZE)

# ------------------------
# Optimize
# ------------------------
model.optimize()

# ------------------------
# Results
# ------------------------
if model.status == GRB.OPTIMAL:
    print(f"\nTotal 20-year cost: {model.objVal:.2f}")

    print("\nYearly line upgrades:")
    for y in years:
        any_up = False
        for (bus_from, bus_to) in lines.keys():
            up = line_upgrade_vars[(bus_from, bus_to, y)].X
            if up > 1e-6:
                if not any_up:
                    print(f"\nYear {y}:")
                    any_up = True
                print(f"  Upgrade on line {bus_from} - {bus_to}: {up:.2f} MW")

    print("\nYearly effective line capacities (cumulative):")
    for y in years:
        print(f"\nYear {y}:")
        for (bus_from, bus_to), data in lines.items():
            base_cap = data['capacity']
            cum_upgrade = sum(
                line_upgrade_vars[(bus_from, bus_to, yy)].X
                for yy in years if yy <= y
            )
            eff_cap = base_cap + cum_upgrade
            print(f"  {bus_from} -> {bus_to}: {eff_cap:.2f} MW  (base {base_cap}, cum. upgrade {cum_upgrade:.2f})")

    print("\nYearly generation (non-zero):")
    for y in years:
        print(f"\nYear {y}:")
        for gen in generators.keys():
            val = gen_vars[(gen, y)].X
            if abs(val) > 1e-6:
                print(f"  {gen}: {val:.2f} MW")

    print("\nYearly load shedding (if any):")
    for y in years:
        any_shed = False
        for load in loads.keys():
            s = shed_vars[(load, y)].X
            if s > 1e-6:
                if not any_shed:
                    print(f"\nYear {y}:")
                    any_shed = True
                print(f"  Shed at {load}: {s:.2f} MW")

else:
    print("Model not optimal. Status code:", model.status)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 24.6.0 24G325)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 580 rows, 560 columns and 5360 nonzeros
Model fingerprint: 0x8c9fe911
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [5e+00, 1e+03]
  Bounds range     [2e+02, 6e+02]
  RHS range        [3e+01, 3e+02]
Presolve removed 141 rows and 180 columns
Presolve time: 0.02s
Presolved: 439 rows, 380 columns, 5926 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     199    4.0028953e+05   0.000000e+00   0.000000e+00      0s

Solved in 199 iterations and 0.03 seconds (0.00 work units)
Optimal objective  4.002895296e+05

Total 20-year cost: 400289.53

Yearly line upgrades:

Year 1:
  Upgrade on line Bus1 - Bus4: 30.00 MW
  Upgrade on line Bus2 - Bus5: